In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from pathlib import Path
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
open_ai_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4.1-nano"
openai = OpenAI()

In [ ]:
knowledge = {}

filenames = glob.glob("./employees/*")

for filename in filenames:
  name = Path(filename).stem.split(' ')[-1]
  with open(filename, "r", encoding="utf-8") as f:
    knowledge[name.lower()] = f.read()

In [ ]:
knowledge.keys()

In [ ]:
knowledge['lancaster']

In [ ]:
filenames = glob.glob("./products/*")

for file in filenames:
  name = Path(file).stem.split(" ")[-1]
  with open(file,'r',encoding = "utf-8") as f:
    knowledge[name] = f.read() 

In [ ]:
knowledge.keys()

In [ ]:
SYSTEM_PREFIX = """
You represent Insurellm, the Insurance Tech company.
You are an expert in answering questions about Insurellm; its employees and its products.
You are provided with additional context that might be relevant to the user's question.
Give brief, accurate answers. If you don't know the answer, say so.

Relevant context:
"""

In [ ]:
def get_relavant_context_simple(message):
  message_arr = message.split()
  message_cleaned = [msg for msg in message_arr if msg.isalpha or msg.isspace]
  context = [knowledge[msg] for msg in message_cleaned if msg in knowledge]
  return context


In [ ]:
from IPython.display import display,Markdown
Markdown(get_relavant_context_simple("thomson")[0])

In [ ]:
def additonal_context(message):
  releavant_context= get_relavant_context_simple(message)
  if not releavant_context:
    result = "No additional info is found about this information"
  else :
    result = "Following info might be useful"
    result = "\n ".join(releavant_context)
  return result

In [ ]:
additonal_context("thomson")

In [ ]:
# In gradio we often use chat with message and history so here it goes

def chat(message,history):
  system_message = SYSTEM_PREFIX + additonal_context(message)
  print(history)
  messages = [{"role":'system','content':system_message}] + history + [{'role':'user','content':message}]
  response = openai.chat.completions.create(model=model,messages=messages)
  return response.choices[0].message.content


In [ ]:
view = gr.ChatInterface(chat).launch(inbrowser=True)